# SciANN Overview

SciANN is a Python library for scientific machine learning that allows you to build neural network models for solving scientific problems, including ordinary and partial differential equations (ODEs and PDEs). SciANN provides a range of classes and tools to help you define and solve scientific problems with neural networks.

## Key Components and Classes

### `sn.Variable`
- Class to define inputs to the network.

### `sn.Field`
- Class to define outputs of the network.

### `sn.Functional`
- Class to construct a nonlinear neural network approximation.

### `sn.Parameter`
- Class to define a parameter for inversion purposes.

### `sn.Data` and `sn.Tie`
- `sn.Data` is used to define the targets when there are observations for any variable.
- `sn.Tie` is used for physical constraints, such as PDEs or equality relations between different variables.

### `sn.SciModel`
- Class to set up the optimization problem, including inputs to the networks, targets (objectives), and the loss function.

### `sn.math`
- Module for mathematical operations, including functions and operations used in defining scientific problems.
- Supports operator overloading, which improves readability when setting up complex mathematical relations, such as PDEs.

## Example Usage

Here's an example of how you can use SciANN to define and solve a scientific problem:

```python
import sciann as sn

# Define variables and fields
x = sn.Variable('x')
t = sn.Variable('t')
u = sn.Functional('u', [x, t], 4*[20], 'tanh')

# Define the PDE
pde = u.diff(t) - u.diff(x, order=2) + sn.math.sin(u)

# Define the optimization problem
model = sn.SciModel([x, t], [pde])

# Compile and train the model
model.compile()
model.train()

# Evaluate the model or make predictions
predictions = model.evaluate()
```
### `Reference`
https://www.sciencedirect.com/science/article/pii/S0045782520307374


In [94]:
import sciann as sn
import numpy as np
import math
import latexify

## Example Usage

Here's an example of how you can use SciANN to define and solve a scientific problem:

$$f(x,y) = sin(x) sin(y), \qquad  x,y \rightarrow [-\pi, \pi] \times [-\pi, \pi]$$
we want to fit a surface, in the form of a neural network, to this dataset. A multi-layer NN approximagting the function $f$ can be constructed as 
$$\hat{f} : (x, y) \mapsto N_f(x, y; \mathbf{W}, \mathbf{b})$$

with the inputs $x$,$y$ and output $\hat{f}$. 


In [2]:
x = sn.Variable("x")
y = sn.Variable("y")
f = sn.Field("f")

A 3-layer neural network with 6 neural units and hyperbolic-tangent activation function can then be constructed as

In [3]:
f = sn.Functional(fields=[f],
                  variables=[x,y],
                  hidden_layers=[6,6,6],
                  activation="tanh")
# Alternatively
# f = sn.Functional("f", [x,y], [6,6,6], "tanh")

At this stage, the parameters of the networks, i.e. set of $\mathbf{W}$, $\mathbf{b}$ for all layers, are randomly initialized. Their current values can be retrieved using the command ``get_weights``:

In [8]:
f.get_weights()

[[array([[-0.23357536,  0.54043233, -0.33839688,  0.3459175 ,  0.23935457,
           0.04551401],
         [-0.34270242,  0.8893965 , -0.2088618 , -0.29452586, -0.16541632,
           0.27990988]], dtype=float32),
  array([ 0.01060783,  0.0034971 , -0.04584777,  0.04082469, -0.03577889,
          0.03148873], dtype=float32)],
 [],
 [array([[-0.40416113, -0.7425922 , -0.910106  , -0.2179015 ,  0.18924825,
           0.15842733],
         [-0.8009313 ,  0.58122617,  0.52707374, -0.52111715, -0.44816196,
           0.82990694],
         [ 0.27064943,  0.22330378, -0.02528486, -0.2791748 , -0.23195536,
          -0.1489158 ],
         [-0.1459364 , -0.32787395,  0.61085075,  0.06647699, -0.79230154,
          -0.72129333],
         [-0.20823024, -0.35128918, -0.05196288,  0.08447584, -0.0602266 ,
          -0.63432765],
         [-0.2800788 ,  0.7174514 , -0.9048397 ,  0.17448181,  0.4176245 ,
          -0.00700031]], dtype=float32),
  array([-0.01759284,  0.01718538, -0.04751943,  0.0066

One can set the parameters of the network to any desired values using the command ``set_weights``.

As another example, a more complex neural network functional as the composition of three blocks, as shown in the figure below <img src="nnImg.jpg" alt="Fig. 1" width="600" height="250">

, can be constructed as

In [9]:
f1 = sn.Functional("f1", [x,y], [4,4], "tanh")
f2 = sn.Functional("f2", [x,y], [4,4], "tanh")
g = sn.Functional("g", [f1,f2], [4,4], "tanh")

In [38]:
weight1 = f1.get_weights()
print(len(weight1))
f1.get_weights()[4]

[array([[-0.17978787],
        [ 0.20236449],
        [-0.16952945],
        [-0.6301778 ]], dtype=float32),
 array([-0.04776594], dtype=float32)]

Any of these functions can be evaluated immediately or after training using the ``eval`` function, by providing discrete data for the inputs:

In [105]:
# Let's create some sample data
# Define the range
x_min = -np.pi
x_max = np.pi
y_min = -np.pi
y_max = np.pi

# Define the number of data points
num_points = 100  # Adjust as needed

# Generate equally spaced values for x and y
x_data = np.linspace(x_min, x_max, num_points)
y_data = np.linspace(y_min, y_max, num_points)

f1_data = np.linspace(x_min, x_max, num_points)
f2_data = np.linspace(y_min, y_max, num_points)

In [103]:
# x_data = sn.Data(x)
# y_data = sn.Data(y)
# f1_data = sn.Data(f1)
# f2_data = sn.Data(f2)

In [106]:
f_test = f.eval([x_data, y_data])
f1_test = f1.eval([x_data, y_data])
f2_test = f2.eval([x_data, y_data])
g_test = g.eval([f1_data, f2_data])


In [77]:
# from numpy import array
# a = array(weight1)
# type(weight1)

Once the networks are initialized, we set up the optimization problem and train the
network by minimizing an objective function, i.e. solving the optimization problem for $\mathbf{W}$
and $\mathbf{b}$. The optimization problem for a data-driven curve-fitting is defined as:
$$\arg\min_{W,b} L(W, b) := \left\| f(x^*, y^*) - Nf(x^*, y^*; W, b) \right\|
$$

where $(x^*, y^*)$ is the set of all discrete points where $f$ is given. For the loss function $\left\| o \right\|$, we use the mean squared-error norm $\left\| o \right\|$ = $\frac{1}{N} {\sum_{x^*,y^* \in I}} (f(x^*, y^*) - \hat{f}(x^*, y^*))^2$.

This problem is set up in SciANN through


In [78]:
m = sn.SciModel(inputs=[x,y],
                targets=[f],
                loss_func="mse",
                optimizer="adam")

The train model is then used to perform the training and identify the parameters of the
neural network:

In [87]:
m.train([x_data, y_data], [f1_data], epochs=400)


Total samples: 100 
Batch size: 64 
Total batches: 2 

Epoch 1/400
2/2 [==============================] - 0s 9ms/step - batch: 0.5000 - size: 50.0000 - loss: 6.1591 - lr: 0.0010 - time: 0.2375
Epoch 2/400
2/2 [==============================] - 0s 3ms/step - batch: 0.5000 - size: 50.0000 - loss: 6.2549 - lr: 0.0010 - time: 0.0928
Epoch 3/400
2/2 [==============================] - 0s 2ms/step - batch: 0.5000 - size: 50.0000 - loss: 6.0410 - lr: 0.0010 - time: 0.1057
Epoch 4/400
2/2 [==============================] - 0s 2ms/step - batch: 0.5000 - size: 50.0000 - loss: 5.9620 - lr: 0.0010 - time: 0.1070
Epoch 5/400
2/2 [==============================] - 0s 2ms/step - batch: 0.5000 - size: 50.0000 - loss: 6.1503 - lr: 0.0010 - time: 0.1091
Epoch 6/400
2/2 [==============================] - 0s 2ms/step - batch: 0.5000 - size: 50.0000 - loss: 6.1149 - lr: 0.0010 - time: 0.1090
Epoch 7/400
2/2 [==============================] - 0s 2ms/step - batch: 0.5000 - size: 50.0000 - loss: 6.0334 - lr: 

Once the training is completed, one can set parameters of a Functional to be trainable
or non-trainable (fixed). For instance, to set $f$ to be non-trainable:


In [88]:
f1.set_trainable(False)

sciann.functionals.mlp_functional.MLPFunctional

The result of this training is shown in Fig. below <img src="nnResult.jpg" alt="Fig. 1" width="600" height="250">, where we have used 400 epochs to perform the training on a dataset generated using a uniform grid of 51 × 51.

Since data was generated from $f(x, y) = \sin(x) \sin(y)$, we know that this is a solution to $\Delta f + 2f = 0$, with $\Delta$ as the Laplacian operator.

As a first illustration of SciANN for physics-informed deep learning, we can constrain the curve-fitting problem with this 'governing equation'. In SciANN, the differentiation operators are evaluated through `sn.math.diff` function. Here, this differential equation can be evaluated as:


In [96]:
@latexify.function
def fxy(x,y):
    return math.sin(x) * math.sin(y)
fxy

In [100]:
from sciann import diff
L = diff(fxy, x, order=2) + diff(fxy, y, order=2) + 2*fxy

AssertionError: Please provide a proper functional object. 